# Current plan

 1. First need to process the pageviews data
    - First group by title
    - Also try grouping by page_id, or assiging a master page id to each title
 2. Then we can start filtering
    - Filter out junk, such as pages with no page id
    - filter out some prefixes such as "Category", "Help", "Portal", "User" etc.
        - Do we keep the "Wikipedia:" prefix?
        - Maybe, maybe not.... it may contain useful information on wikipedia itself...
    - Filter based on pageviews, some sort of quantile based approach? Keep top K?

### Next steps

 - After grouping the data, we should try to join it with the parsed wikipedia data dump..  
 - It will be easier to filter out the junk pages that way, can then also filter by length of the page content.

In [1]:
import pandas as pd


pageviews_file = "/Users/einar/git/hafsteinn/together_rag/data/wikipedia_meta/aggregated/is.wikipedia_monthly_views.parquet"
df_pageviews = pd.read_parquet(pageviews_file)
# print(len(df_pageviews))

print(f"Number of pageviews rows: {len(df_pageviews):,}")

df_pageviews.head(2)


Number of pageviews rows: 1,182,149


,wiki_code,article_title,views,page_ids
0,is.wikipedia,"""\""Weird_Al\""_Yankovic""",1,[52652]
1,is.wikipedia,$,2,[30714]


In [4]:
df_pageviews.article_title.value_counts()

article_title
Skóggangur                                              12
Flóðið                                                  12
Flórídastraumurinn                                      12
Flórídasund                                             12
Flóttamannastofnun_Sameinuðu_þjóðanna                   12
                                                        ..
Mynd:Potlatch_gas.jpg                                    1
Flokkur:Skautafélög_eftir_löndum                         1
Mynd:Praia_do_Mucuripe.jpg                               1
Mynd:President_Megawati_Sukarnoputri_-_Indonesia.jpg     1
汉语/漢語                                                    1
Name: count, Length: 202415, dtype: int64

In [5]:
df_pageviews[df_pageviews.article_title == "Skóggangur"]

,wiki_code,article_title,views,page_ids
101207,is.wikipedia,Skóggangur,7,[80529]
205549,is.wikipedia,Skóggangur,3,[80529]
315660,is.wikipedia,Skóggangur,5,[80529]
412546,is.wikipedia,Skóggangur,6,[80529]
503532,is.wikipedia,Skóggangur,6,[80529]
589230,is.wikipedia,Skóggangur,1,[80529]
688970,is.wikipedia,Skóggangur,1,[80529]
784954,is.wikipedia,Skóggangur,7,[80529]
871163,is.wikipedia,Skóggangur,7,"[80529, 80528]"
965392,is.wikipedia,Skóggangur,10,[80529]


In [6]:
assert(len(df_pageviews.wiki_code.unique()) == 1)

In [29]:
language = df_pageviews.wiki_code.unique()[0].split(".")[0]

URL_TEMPLATE = "https://{language}.wikipedia.org/w/index.php?curid={page_id}"

## First try group by title

Sum the views column, combine the page_ids, should probably create a set instead of lists

In [11]:
from IPython.display import display


def union_page_ids(x):
    return set().union(*x)

df_grouped = df_pageviews.groupby("article_title").agg({
    "views": "sum", 
    "page_ids": union_page_ids
})

df_grouped = df_grouped.sort_values("views", ascending=False)

print(f"N df_grouped: {len(df_grouped)}")

display(df_grouped[df_grouped.index == "Skóggangur"])

df_grouped.sort_values("views", ascending=False).head(2)

N df_grouped: 202415


,views,page_ids
article_title,,
Skóggangur,69,"{80528, 80529}"


,views,page_ids
article_title,,
XXX_Rottweilerhundar,972644,"{87556, null}"
Forsíða,435634,"{4, null}"


In [16]:
df_grouped.describe()

,views
count,202415.000000
mean,90.057021
std,2492.686175
min,1.000000
25%,2.000000
50%,7.000000
75%,37.000000
max,972644.000000


## Should we try a percentile based approach?

In [33]:
def first_not_null_page_id(x):
    return next((page_id for page_id in x if page_id is not None or page_id != "null"), None)

df_grouped["percentile"] = df_grouped.views.rank(pct=True)
df_grouped["url"] = df_grouped.apply(lambda x: URL_TEMPLATE.format(language=language, page_id=first_not_null_page_id(x.page_ids)) if x.page_ids else None, axis=1)

df_grouped.sort_values("percentile", ascending=True).head(3)

,views,page_ids,percentile,url
article_title,,,,
Notandi:Okin,1,{null},0.103253,https://is.wikipedia.org/w/index.php?curid=null
Notandi:Oursana,1,{null},0.103253,https://is.wikipedia.org/w/index.php?curid=null
Σπάρτη,1,{28321},0.103253,https://is.wikipedia.org/w/index.php?curid=28321


### First remove all pages with {null} in the page_ids column



In [38]:
df_grouped = df_grouped[df_grouped.page_ids != set(["null"])]
print(f"N df_grouped after removing: {len(df_grouped)}")
df_grouped.sort_values("percentile", ascending=True).head(3)

N df_grouped after removing: 137564


,views,page_ids,percentile,url
article_title,,,,
Spjall:Mateusz_Morawiecki,1,{165574},0.103253,https://is.wikipedia.org/w/index.php?curid=165574
Jpop,1,{118304},0.103253,https://is.wikipedia.org/w/index.php?curid=118304
Winnie_Madikizela-Mandela,1,{158760},0.103253,https://is.wikipedia.org/w/index.php?curid=158760


In [61]:
# extract all unique prefixes before ":" in the article_title (index)
# not all titles have a prefix
from collections import defaultdict 

prefixes = defaultdict(int)

for title in df_grouped.index:
    if ":" in title:
        prefix = title.split(":")[0]
        prefixes[prefix] += 1

df_prefixes = pd.Series(prefixes)
df_prefixes = df_prefixes.sort_values(ascending=False)
df_prefixes.head(25)

Flokkur             19229
Spjall               7500
Snið                 4176
Notandaspjall        3084
Notandi              2791
Mynd                 2504
Wikipedia             921
Talk                  814
Module                477
Wikipediaspjall       323
Sniðaspjall           241
Flokkaspjall          212
Melding               205
is                    194
flokkur               134
Hjálp                  68
User                   38
wikipedia              34
notandi                31
Listi_yfir_CSI         31
snið                   24
Gátt                   24
Myndaspjall            23
Hjálparspjall          22
Thomas_&_Friends       18
dtype: int64

In [65]:
prefixes_to_remove = [
   "Flokkur:",
   "Spjall:",
   "Snið:",
   "Notandaspjall:",
   "Notandi:",
   "Mynd:",
   "Wikipedia:",
   "Talk:",
   "Module:",
   "Wikipediaspjall:",
   "Sniðaspjall:",
   "Flokkaspjall:",
   "Melding:",
   #: "is",
   "flokkur:",
   "Hjálp:",
   "User:",
   "wikipedia:",
   "notandi:",
   "Listi_yfir_CSI:",
   "snið:",
   "Gátt:",
   "Myndaspjall:",
   "Hjálparspjall:",
]

In [64]:
for prefix in df_prefixes.index[20:22]:
    display(df_grouped[df_grouped.index.str.contains(prefix + ":")].sort_values("views", ascending=False).head(3))
    print()

,views,page_ids,percentile,url
article_title,,,,
snið:cite_conference,8,{170632},0.525309,https://is.wikipedia.org/w/index.php?curid=170632
snið:cite_journal,6,{44777},0.475768,https://is.wikipedia.org/w/index.php?curid=44777
snið:tl,4,{35331},0.400294,https://is.wikipedia.org/w/index.php?curid=35331


,views,page_ids,percentile,url
article_title,,,,
Gátt:Úrvalsefni,5598,{113259},0.999279,https://is.wikipedia.org/w/index.php?curid=113259
Gátt:Tölvuleikir,578,{40558},0.972621,https://is.wikipedia.org/w/index.php?curid=40558
Gátt:Landafræði,448,{63107},0.962977,https://is.wikipedia.org/w/index.php?curid=63107


In [67]:
# remove all from df_grouped where the index contains any of the prefixes in prefixes_to_remove

for prefix in prefixes_to_remove:
    df_grouped = df_grouped[~df_grouped.index.str.startswith(prefix)]


print(f"N df_grouped after removing: {len(df_grouped)}")


N df_grouped after removing: 94658


## Percentile based approach

In [68]:
df_grouped[df_grouped.percentile > 0.1].sort_values("views", ascending=True)

,views,page_ids,percentile,url
article_title,,,,
Porphyrio_martinica,1,{122653},0.103253,https://is.wikipedia.org/w/index.php?curid=122653
Alexander_hinn_mikli,1,{23323},0.103253,https://is.wikipedia.org/w/index.php?curid=23323
Alive_festival,1,{115250},0.103253,https://is.wikipedia.org/w/index.php?curid=115250
Aljóðasamtök_kommúnista,1,{27687},0.103253,https://is.wikipedia.org/w/index.php?curid=27687
Alkani,1,{52959},0.103253,https://is.wikipedia.org/w/index.php?curid=52959
...,...,...,...,...
Listi_yfir_íslensk_póstnúmer,49968,"{5127, null}",0.999965,https://is.wikipedia.org/w/index.php?curid=5127
Fiann_Paul,53030,"{134609, null}",0.999970,https://is.wikipedia.org/w/index.php?curid=134609
Carles_Puigdemont,206671,"{143597, null}",0.999990,https://is.wikipedia.org/w/index.php?curid=143597


# Have now removed much of the "junk" from the pageviews data, now on to the actual data



In [69]:
df_dump = pd.read_parquet("iswiki.processed_datadump.parquet")

print(len(df_dump))
df_dump.head()

86163


,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,raw_text
0,3,Finnland,https://is.wikipedia.org/wiki/Finnland,4306,553,0,,110,47,2024-04-18T10:18:03Z,"\nFinnland (finnska Suomi; sænska: Finland), f...",{{Land\n| nafn = Lýðveldið Finnland\n| nafn_á_...
1,4,Forsíða,https://is.wikipedia.org/wiki/Fors%C3%AD%C3%B0a,28,5,0,,11,0,2021-01-02T00:52:18Z,\n\n\n\n\n\n\n\nGrein mánaðarins\n\nEldri grei...,"<templatestyles src=""Template:Forsíða/styles.c..."
2,6,Reykjavík,https://is.wikipedia.org/wiki/Reykjav%C3%ADk,2323,309,0,,51,41,2024-04-09T12:51:22Z,"\n\nReykjavík er höfuðborg Íslands, fjölmennas...",{{Um|höfuðborg Íslands|aðrar merkingar orðsins...
3,7,Svíþjóð,https://is.wikipedia.org/wiki/Sv%C3%AD%C3%BEj%...,3025,360,0,,44,56,2024-08-12T16:35:20Z,"\n\nSvíþjóð (sænska: Sverige), formlegt heiti ...",{{Land \n|nafn = Konungsríkið Svíþjóð\n|nafn_á...
4,591,Danmörk,https://is.wikipedia.org/wiki/Danm%C3%B6rk,4711,489,0,,55,58,2024-07-18T11:56:58Z,\nDanmörk (danska: Danmark; ) er land í Norður...,{{Land\n| nafn = Konungsríkið Danmörk\n| nafn_...


## join strategy

I need to try joining on both title and page_id... 

Lets start by title, and see which pages still are not joined..

In [71]:
df_pageviews["title_match"] = df_pageviews.article_title.str.replace("_", " ").str.lower()
df_dump["title_match"] = df_dump.title.str.replace("_", " ").str.lower()

data_titles = set(df_dump.title_match)
pageviews_titles = set(df_pageviews.title_match)

print(f"Number of titles in data: {len(data_titles)}")
print(f"Number of titles in pageviews: {len(pageviews_titles)}")
print(f"Number of titles in both data: {len(data_titles.intersection(pageviews_titles))}")
print(f"Number of unmatched titles in data: {len(data_titles) - len(data_titles.intersection(pageviews_titles))}")
print(f"Number of unmatched titles in pageviews: {len(pageviews_titles) - len(data_titles.intersection(pageviews_titles))}")

Number of titles in data: 85506
Number of titles in pageviews: 200264
Number of titles in both data: 85148
Number of unmatched titles in data: 358
Number of unmatched titles in pageviews: 115116


In [72]:
pd.set_option("display.max_colwidth", 100)
df_dump[~df_dump.title_match.isin(df_pageviews.title_match)].sort_values(by=["external_link_count"], ascending=False).head(5)



,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,raw_text,title_match
2991,10824,Grunneiningar SI kerfisins,https://is.wikipedia.org/wiki/Grunneiningar_SI_kerfisins,3,1,0,,0,0,2005-07-22T01:15:54Z,#tilvísun SI grunneining,#tilvísun [[SI grunneining]],grunneiningar si kerfisins
44306,101663,Chub Makríll,https://is.wikipedia.org/wiki/Chub_Makr%C3%ADll,3,1,0,,0,0,2011-10-02T22:44:00Z,#tilvísun Spænskur makríll,#tilvísun [[Spænskur makríll]],chub makríll
47415,110072,"Alfreð Clausen - Lagið úr ""Rauðu myllunni""",https://is.wikipedia.org/wiki/Alfre%C3%B0_Clausen_-_Lagi%C3%B0_%C3%BAr_%22Rau%C3%B0u_myllunni%22,100,12,0,,2,0,2013-02-17T11:25:27Z,\nAlfreð Clausen syngur erlend lög er 78-snúninga hljómplata gefin út af Íslenzkum tónum árið 19...,{{Breiðskífa\n|Nafn = Alfreð Clausen syngur erlend lög\n|Gerð = IM 3...,"alfreð clausen - lagið úr ""rauðu myllunni"""
47152,109317,14 barnalög,https://is.wikipedia.org/wiki/14_barnal%C3%B6g,10,1,0,,0,0,2012-08-25T16:12:04Z,#tilvísun Jan Morávek og hljómsveit ásamt barnakór flytja 14 barnalög,#tilvísun [[Jan Morávek og hljómsveit ásamt barnakór flytja 14 barnalög]],14 barnalög
47147,109312,Alfreð Clausen og Sigrún Ragnars - Hvað er svo glatt,https://is.wikipedia.org/wiki/Alfre%C3%B0_Clausen_og_Sigr%C3%BAn_Ragnars_-_Hva%C3%B0_er_svo_glatt,5,1,0,,0,0,2012-08-25T16:03:30Z,#tilvísun Hvað er svo glatt,#tilvísun [[Hvað er svo glatt]],alfreð clausen og sigrún ragnars - hvað er svo glatt


### Need to cleanup the df_dump a bit, 

Remove duplicate records based on url, (first set url to lower case)

In [78]:
df_dump[df_dump.title_match == "visa-bikarinn"]

,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,raw_text,title_match
18983,43536,VISA-bikarinn,https://is.wikipedia.org/wiki/VISA-bikarinn,60,9,0,,1,0,2007-05-17T21:16:43Z,VISA-bikarinn er bikarkeppni sem er keppt er í knattspyrnu bæði í karla- og kvennaflokki einnig ...,'''VISA-bikarinn''' er [[bikarkeppni]] sem er keppt er í [[knattspyrna|knattspyrnu]] bæði í karl...,visa-bikarinn
18988,43544,VISA-Bikarinn,https://is.wikipedia.org/wiki/VISA-Bikarinn,2,1,0,,0,0,2007-05-17T23:05:32Z,#tilvísun VISA-bikarinn,#tilvísun [[VISA-bikarinn]],visa-bikarinn
18989,43545,Visa-bikarinn,https://is.wikipedia.org/wiki/Visa-bikarinn,2,1,0,,0,0,2007-05-17T23:06:30Z,#tilvísun VISA-bikarinn,#tilvísun [[VISA-bikarinn]],visa-bikarinn
18992,43549,Visa-Bikarinn,https://is.wikipedia.org/wiki/Visa-Bikarinn,2,1,0,,0,0,2007-05-17T23:13:25Z,#tilvísun VISA-bikarinn,#tilvísun [[VISA-bikarinn]],visa-bikarinn


In [79]:
df_merged = df_dump.merge(df_pageviews, left_on="title_match", right_on="title_match", how="left")

print(f"n in dump: {len(df_dump):,}")
print(f"n in merged: {len(df_merged):,}")
# print("n missing from merged: ", len(df_dump) - len(df_merged))


n in dump: 86,163
n in merged: 817,802
